In [2]:
import torch
import contractions
import unidecode
import re
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import load_dataset
import pandas as pd


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = T5Tokenizer.from_pretrained('castorini/doc2query-t5-base-msmarco')
Queriesmodel = T5ForConditionalGeneration.from_pretrained('castorini/doc2query-t5-base-msmarco')
Queriesmodel.to(device)

2023-05-15 09:44:57.522578: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-15 09:44:58.052640: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-12.0/lib64:/usr/local/cuda-12.0/lib64
2023-05-15 09:44:58.052687: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-12.0/lib64:/usr/local/cuda-12.0/lib64
2023-05-15 09:44:58.

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│    9                                                                                             │
│   10 device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')                       │
│   11                                                                                             │
│ ❱ 12 tokenizer = T5Tokenizer.from_pretrained('castorini/doc2query-t5-base-msmarco')              │
│   13 Queriesmodel = T5ForConditionalGeneration.from_pretrained('castorini/doc2query-t5-base-m    │
│   14 Queriesmodel.to(device)                                                                     │
│   15                                                                                             │
│                                                                                                  │
│ /home/g6/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1770 in      │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   1767 │   │   │   │   elif is_remote_url(file_path):                                            │
│   1768 │   │   │   │   │   resolved_vocab_files[file_id] = download_url(file_path, proxies=prox  │
│   1769 │   │   │   else:                                                                         │
│ ❱ 1770 │   │   │   │   resolved_vocab_files[file_id] = cached_file(                              │
│   1771 │   │   │   │   │   pretrained_model_name_or_path,                                        │
│   1772 │   │   │   │   │   file_path,                                                            │
│   1773 │   │   │   │   │   cache_dir=cache_dir,                                                  │
│                                                                                                  │
│ /home/g6/.local/lib/python3.8/site-packages/transformers/utils/hub.py:409 in cached_file         │
│                                                                                                  │
│    406 │   user_agent = http_user_agent(user_agent)                                              │
│    407 │   try:                                                                                  │
│    408 │   │   # Load from URL or cache if already cached                                        │
│ ❱  409 │   │   resolved_file = hf_hub_download(                                                  │
│    410 │   │   │   path_or_repo_id,                                                              │
│    411 │   │   │   filename,                                                                     │
│    412 │   │   │   subfolder=None if len(subfolder) == 0 else subfolder,                         │
│                                                                                                  │
│ /home/g6/.local/lib/python3.8/site-packages/huggingface_hub/utils/_validators.py:120 in          │
│ _inner_fn                                                                                        │
│                                                                                                  │
│   117 │   │   if check_use_auth_token:                                                           │
│   118 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   119 │   │                                                                                      │
│ ❱ 120 │   │   return fn(*args, **kwargs)                                                         │
│   121 │                                                    

In [11]:
df_docs=pd.read_csv('/home/g6/Desktop/Fawzy/ag_news_csv/AG_News.csv')
df_docs.head()
df_docs['title'].duplicated().sum()

125636

In [4]:
def preprocess(doc):
    doc = unidecode.unidecode(doc) # transliterates any unicode string into the closest possible representation in ascii text.
    doc = contractions.fix(doc) # expands contractions                   
    doc = re.sub('[\t\n]', ' ', doc) # remove newlines and tabs
    doc = re.sub(r'@[A-Za-z0-9_]+', '', doc) # remove mentions
    doc = re.sub(r'#[A-Za-z0-9_]+', '', doc) #remove hashtags
    doc = re.sub(r'https?://[^ ]+', '', doc)
    doc = re.sub(r'www.[^ ]+', '', doc)
    doc = re.sub('[^A-Za-z]+', ' ', doc) # remove all characters other than alphabet
    doc = re.sub(' +', ' ', doc) # substitute any number of space with one space only
    doc = doc.strip() # remove spaces from begining and end and lower the text
    return doc

In [5]:
df_docs.shape

(240000, 4)

In [6]:
df_docs= df_docs[df_docs['description']!=""]
df_docs= df_docs[df_docs['title']!=""]

In [7]:
df_docs['description']= df_docs['description'].apply(lambda x: preprocess(x))
df_docs['title']= df_docs['title'].apply(lambda x: preprocess(x))


In [ ]:
df.drop

In [ ]:
from sentence_transformers import  SentenceTransformer
corpus= df_docs['description'].tolist()
model = SentenceTransformer('sentence-transformers/msmarco-MiniLM-L6-cos-v5')
corpusVectors= model.encode(corpus)

In [ ]:
corpusVectors.shape

(240000, 384)

In [ ]:
import h5py
with h5py.File('AG_News.hdf5', 'w') as f:
    f.create_dataset('train', data=corpusVectors)
    f.attrs['distance']= 'angular'

### News vs. Queries

In [ ]:
in__= df_docs['description'].tolist()

In [ ]:
df_docs['description'].isnull().sum()

0

In [ ]:
def get_query(doc):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  input_ids = tokenizer.encode(doc, return_tensors='pt').to(device)
  my_set = set()
  outputs = Queriesmodel.generate(
      input_ids=input_ids,
      max_length=2000,
      do_sample=True,
      top_k=10,
      num_return_sequences=20)

  for i in range(3):
      my_set.add(tokenizer.decode(outputs[i], skip_special_tokens=True))
  return ". ".join(my_set)

In [ ]:
import csv
with open('AG_News_Queries.csv', mode='w', newline='', encoding= 'utf-8') as file:
    writer= csv.writer(file)
    writer.writerow(['Document', 'Query'])
    
    for doc in in__:
        row= (doc, get_query(doc))
        writer.writerow(row)
        torch.cuda.empty_cache()

In [ ]:
df_queries= pd.read_csv('AG_News_Queries.csv')
df_queries.head()

,Document,Query
0,Reuters Short sellers Wall Street s dwindling ...,who are the cynics on reuters. what is the gre...
1,Reuters Private investment firm Carlyle Group ...,who is carlyle. who is carlyle group. carlyle ...
2,Reuters Soaring crude prices plus worries abou...,does a rising oil price affect market share. w...
3,Reuters Authorities have halted oil export flo...,why is pipeline shut in southern iraq. who has...
4,AFP Tearaway world oil prices toppling records...,what are the oil prices?. what is the oil price


In [ ]:
df_queries.drop_duplicates()

,Document,Query
0,Reuters Short sellers Wall Street s dwindling ...,who are the cynics on reuters. what is the gre...
1,Reuters Private investment firm Carlyle Group ...,who is carlyle. who is carlyle group. carlyle ...
2,Reuters Soaring crude prices plus worries abou...,does a rising oil price affect market share. w...
3,Reuters Authorities have halted oil export flo...,why is pipeline shut in southern iraq. who has...
4,AFP Tearaway world oil prices toppling records...,what are the oil prices?. what is the oil price
...,...,...
76870,RAMALLAH West Bank October No crowds of well w...,which u.s. state has been responsible for yass...
76871,For all the focus on foreign policy in this ca...,who is the president of iran today?. what is t...
76872,Security expert express concerns that cash mac...,what could happen if cash machines get viruses...
76873,Sixteen workers are critically hurt in a huge ...,where is oil refinery booleans located. where ...


In [ ]:
queries= df_queries['Query'].tolist()

In [ ]:
from sentence_transformers import  SentenceTransformer
model = SentenceTransformer('sentence-transformers/msmarco-MiniLM-L6-cos-v5')

In [ ]:
#grt distinct corpus
queriesVectors= model.encode(queries)

In [ ]:
queriesVectors.shape

(76875, 384)

In [ ]:
with h5py.File('AG_News.hdf5', 'w') as f:
    f.create_dataset('test', data=queriesVectors)
    f.attrs['distance']= 'angular'

In [ ]:
indicesList=[]
distances=[]
def search(queryVector, corpusVectors):
    from sklearn.neighbors import NearestNeighbors
    nbrs = NearestNeighbors(n_neighbors=100, algorithm='brute').fit(corpusVectors)
    _, indices = nbrs.kneighbors([queryVector])
    indicesList.append(indices)
    distances.append(_)

In [ ]:
import numpy as np

for q in queriesVectors:
    search(q, corpusVectors)
    
df_indices= pd.DataFrame(np.concatenate(indicesList), columns=[f"column_{i+1}" for i in range (100)])
df_distances= pd.DataFrame(np.concatenate(distances), columns=[f"column_{i+1}" for i in range (100)])

In [ ]:
#df_distances.to_csv('distances.csv', index=False)
#df_indices.to_csv('indices.csv', index=False)

df_distances=pd.read_csv('distances.csv')
df_indices= pd.read_csv('neighbors.csv')

In [ ]:
df_distances_arr= df_distances.values
df_indices_arr= df_indices.values

In [ ]:
# with h5py.File('AG_News.hdf5', 'w') as f:
#     f.create_dataset('train', data=corpusVectors)
#     f.create_dataset('test', data=queriesVectors)
#     f.create_dataset('neighbors', data=df_indices_arr)
#     f.create_dataset('distances', data=df_distances_arr)
#     f.attrs['distance']= 'angular'

In [ ]:
with h5py.File('AG_News.hdf5', 'r') as f:
    d=f['test']
    print(d[0].shape)


(384,)


In [ ]:
l= "New york Accidents"
r= model.encode(l)
search(r, corpusVectors)

In [ ]:
indicesList[0][0]

array([ 73170, 193170,  80340, 200340,  16231, 136231,  40350, 160350,
        33770, 153770,  47053, 167053, 127551,   7551, 134874,  14874,
        99119, 219119, 133900,  13900, 204119,  84119, 205150,  85150,
       153217,  33217, 168486,  48486, 136020,  16020, 204381,  84381,
       110730, 230730, 178668,  58668, 219110,  99110,  48662, 168662,
        17389, 137389, 201539,  81539,   4957, 124957, 131714,  11714,
       158047,  38047, 209623,  89623, 165897,  45897, 137452,  17452,
        84160, 204160, 176383,  56383, 168545,  48545, 188671,  68671,
       159163,  39163,  66290, 186290,  46788, 166788, 125122,   5122,
       103052, 223052, 201587,  81587,  71132, 191132,  62905, 182905,
         3322, 123322,  20931, 140931, 203616,  83616,  84377, 204377,
        56740, 176740,  36569, 156569, 223075, 103075, 194164,  74164,
        76079, 196079,  91960, 211960])

In [ ]:
df1= pd.read_csv('/home/g6/Desktop/Fawzy/AG_News_Queries.csv')
df1.head()

NameError: name 'pd' is not defined